## 2.3 Object Detection Inferencing

### 2.3.1 <font color="orange">Training vs Inferencing</font> Deep Learning Model
- **Training** memerlukan **dataset** dan akan menghasilkan **Model**.
- **Inference** memerlukan **Model** dan **data test** yang akan menghasilkan **Prediction**.  
<img src="resource/training-inferencing.jpg" style="width:700px"></img><br><br>

### 2.3.2 Intro OpenCV DNN

**OpenCV DNN - Deep Neural Network** adalah library untuk **Inference** atau **Forward Pass** Model Deep Learning dari beragam framework populer. Menyediakan struktur prrogram yang sederhana dan high performance (mensupport beragam CPU,GPU dan Inference Engine).
- Compatibility : > OpenCV 3.3
- Wiki : https://github.com/opencv/opencv/wiki/Deep-Learning-in-OpenCV
- The supported frameworks:
    - Caffe
    - TensorFlow
    - Torch
    - Darknet (Yolo)
    - Models in ONNX format

- Load Deep Learning Model using OpenCV DNN
    - `cv2.dnn.readNet(model, configration)` 
    - where :
        - `model` :
            - `*.caffemodel` (Caffe, http://caffe.berkeleyvision.org/)
            - `*.pb` (TensorFlow, https://www.tensorflow.org/)
            - `*.t7` | `*.net` (Torch, http://torch.ch/)
            - `*.weights` (Darknet, https://pjreddie.com/darknet/)
        - `configuration` :
            - `*.prototxt` (Caffe, http://caffe.berkeleyvision.org/)
            - `*.pbtxt` (TensorFlow, https://www.tensorflow.org/)
            - `*.cfg` (Darknet, https://pjreddie.com/darknet/)
    - This function automatically detects an origin framework of trained model and calls an appropriate function such 
        - `cv2.dnn.readNetFromCaffe` 
        - `cv2.dnn.readNetFromTensorflow`
        - `cv2.dnn.readNetFromTorch` 
        - `cv2.dnn.readNetFromDarknet`
    - OpenCV DNN config file bisa ditemukan [disini](https://github.com/opencv/opencv_extra/tree/4.x/testdata/dnn)

### 2.3.3 Inferencing Yolo V8 - PyTorch Model using OpenCV DNN
- To use Pytorch Model in OpenCV DNN we need to convert the Pytorch model data type `.pt` into ONNX format.
- We will use [Convert Pytorch Model (.pt) to ONNX.ipynb](https://colab.research.google.com/drive/1IDHaSJyIauPgI_TE9UXHLm2m2nUUaKXb) Notebook file in [Google Colab](https://colab.research.google.com/)
- On that notebook we will download Pytorch YOLOv8 small model (`yolov8s.pt`) then converting it into ONNX format `yolov8s.onnx`
- Don't forget to put the downloaded file (`.onnx`) into the `model/` folder.

- Downgrade OpenCV to version 4.7.x
    - This downgrade is required due to Issue related to ONNX YoloV8 in OpenCV DNN for version >= 4.8.x
        - https://github.com/ultralytics/ultralytics/issues/1836
    - Open Anaconda Prompt 
    - Activate environment
    ```
    conda activate BelajarOpenCV
    ```
    - Downgrade to OpenCV 4.7.x
    ```
    pip install --force-reinstall opencv-python==4.7.0.72 --user
    ```


In [ ]:
import cv2 
import numpy as np

In [ ]:
cv2.__version__

- Model `yolov8s` di training dengan `COCO Dataset` yang terdiri dari `80 class names` dalam `80 class index`
- load yolo coco class names via `.load_coco_class_names_yolo()` in `coco.py`

In [ ]:
import coco

classNames = coco.load_coco_class_names_yolo()

print(classNames)

- load yolo model
    - pastikan `yolov8s.onnx` sudah ada dalam folder `model/`

In [ ]:
# load model
model = "model/yolov8s.onnx"
net = cv2.dnn.readNetFromONNX(model)

- load image and convert to blob with `scaleFactor=1/255.0`

In [ ]:
img = cv2.imread("image1.jpg")

resize_h, resize_w = 320, 320 

blob = cv2.dnn.blobFromImage(img, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)

- do a net forward (inferencing)

In [ ]:
net.setInput(blob)
output = net.forward()

In [ ]:
output[0].shape

- Postprocessing detection result via `utils.py`
    - Apply [NMS Box](https://learnopencv.com/tag/cv-dnn-nmsboxes/)
    - Draw Detection Box
    - use `.postprocess` to do postprocess

In [ ]:
import utils 
utility = utils.Utils()

# do a postprocessing
img = utility.postprocess_onnx(output, img, classNames, confThreshold = 0.5, nmsThreshold = 0.3, font_size=0.5, 
                        color=(255,127,0), text_color=(255,255,255), input_size=[resize_h, resize_w])

- show result

In [ ]:
cv2.imshow("detection result", img)
cv2.waitKey()
cv2.destroyAllWindows()

- Inverencing from video file for Yolo - Pytorch model

In [ ]:
# load model
model = "model/yolov8s.onnx"
net = cv2.dnn.readNetFromONNX(model)

# load video
cap = cv2.VideoCapture("video.mp4")
           
# iterate for each frame in video
while cap.isOpened():
    
    # get image on each frame
    ret, frame = cap.read()
    if not ret:
        break
    h, w, c = frame.shape
    resize_h, resize_w = 320, 320
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (resize_w, resize_h), (0, 0, 0), swapRB=True, crop=False)

    # do a net forward (inferencing)
    net.setInput(blob)
    output = net.forward()

    # do a postprocessing
    frame = utility.postprocess_onnx(output, frame, classNames, confThreshold = 0.5, nmsThreshold = 0.3, font_size=0.5, 
                            color=(255,127,0), text_color=(255,255,255), input_size=[resize_h, resize_w])

    # show result
    cv2.imshow('Frame',frame)

    # wait 25ms per frame and close using 'q' 
    if cv2.waitKey(1) == ord('q'):
          break


cap.release()
cv2.destroyAllWindows()